In [ ]:
from src.esn import ESN
from src.utils import config, helper
from src.conceptors import compute_conceptor, loading_ridge_report, compute_conceptor_diag, ridge_regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Diagonal conceptor

In [ ]:
N = 400

In [ ]:
prng = np.random.default_rng(1001)

In [ ]:
esnConfig = config.ESNConfig(
    input_size=1,
    reservoir_size= N,
    output_size=1,
    feedback=False,
    spectral_radius=1,
    init_weights='normal',
    init_weights_b='normal',
    init_weights_in='normal',
    init_weights_density=10/N,
    init_weights_in_density=1.0,
    init_weights__args={'loc': 0., 'scale': 1.},
    init_weights_b__args={'loc': 0., 'scale': 0.2},
    init_weights_in__args={'loc': 0., 'scale': 1.},
)

In [ ]:
esn = ESN(esnConfig, prng)

In [ ]:
T_pattern = 5000
ut = [
    #helper.n_periodic(5, T_pattern, np.random.default_rng(27)),
    #helper.n_periodic(5, T_pattern, np.random.default_rng(42)),
    helper.n_periodic_jong(0, 5, T_pattern),
    helper.n_periodic_jong(1, 5, T_pattern),
    helper.n_sine(8.83, T_pattern, phase=0),
    helper.n_sine(9.83, T_pattern, phase=0.)
]

In [ ]:
T_WASHOUT = 200
T_ADAPT = 500
T_LEARN = 5000 - 200 - 500
LOADING_REGULARIZER = 1e-3
APERTURE = 8

In [ ]:
# Diagonal "trick": initial biais of the input driven reservoir with random conceptor
C_diag = []
for i in range (4):
    diag = prng.uniform(size=(N))
    C_diag.append(np.diag(diag))

## Adaptation (change C)

In [ ]:
x_init = prng.uniform(low=-1., high=1., size=(esn.reservoir_size, 1))
xt_conceptor, yt_conceptor = list(zip(*[
    esn.harvest_states(ut[i][:T_WASHOUT+T_ADAPT].copy(), x_init=x_init.copy(), C=C_diag[i])
    for i in range(len(ut))
]))

In [ ]:
# Recompute the conceptor, but restrict it to a diagonale matrix (cf. eq 38 Jong (2021))
Ci = [
    
    compute_conceptor_diag(xt_i[T_WASHOUT:, :], aperture=APERTURE)
    for xt_i in xt_conceptor
]

In [ ]:
# Monitor the scaling values of C
fig, ax = plt.subplots(len(ut), 1, figsize=(12, len(ut)), sharex=True, sharey=True)
for i in range(len(ut)):
    ax[i].plot(np.sort(Ci[i][:,0]))
plt.tight_layout()

## Learning (Wout and W with C_diag on the loop)

In [ ]:
xt_conceptor, yt_conceptor = list(zip(*[
    esn.harvest_states(ut[i][T_WASHOUT+T_ADAPT:].copy(), x_init=xt_conceptor[i][[-1],:].T.copy(), C=Ci[i])
    for i in range(len(ut))
]))

In [ ]:
ut_learn = [ut[i][T_WASHOUT+T_ADAPT:] for i in range (len(ut))]

In [ ]:
X = helper.concatenate_patterns(xt_conceptor, 1)
U = helper.concatenate_patterns(ut_learn, 1)
Y_T = U.copy() 

X_ = helper.concatenate_patterns(xt_conceptor, 1, shift=-1)
B = np.repeat(esn.b, X_.shape[0], axis=1).T

In [ ]:
#W_loaded = loading_ridge_report(X, X_, B, regularizer=LOADING_REGULARIZER), good exercise to do it like that also! but more complex

W_loaded = ridge_regression(X_, (np.dot(esn.w, X_.T) + np.dot(esn.w_in, U.T)).T, regularizer=LOADING_REGULARIZER)

In [ ]:
w_before_loading = esn.w.copy()
esn.w = W_loaded.copy()

In [ ]:
# W_out is not yet learned, the output is not reflecting the input (cf. scale)
fig, ax = plt.subplots(len(ut), 1, figsize=(12, len(ut)), sharex=True, sharey=True)
for i in range(len(ut)):
    ax[i].plot(ut[i][:50], ls='--')
    ax[i].plot(yt_conceptor[i][:50])
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(len(ut), 1, figsize=(12, len(ut)), sharex=True, sharey=True)
for i in range(len(ut)):
    ax[i].plot(ut[i][:30], ls='--')
    ax[i].plot(xt_conceptor[i][:30][:,1:10])
plt.tight_layout()

## Plot

In [ ]:
esn.update_weights(X, U, Y_T, alpha=1e-3)

In [ ]:
ut_zero = np.zeros_like(ut[0])
x_init = prng.uniform(low=-1., high=1., size=(esn.reservoir_size, 1))
xt_trained, yt_trained = list(zip(*[
    esn.harvest_states(ut_zero.copy(), x_init=x_init.copy(), C=Ci[i])
    for i in range(len(ut))
]))

In [ ]:
fig, ax = plt.subplots(len(ut), 1, figsize=(12, len(ut)), sharex=True, sharey=True)
for i in range(len(ut)):
    ax[i].plot(ut[i][50:90], ls='--')
    ax[i].plot(yt_trained[i][50:90])
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(len(ut), 1, figsize=(12, len(ut)), sharex=True, sharey=True)
for i in range(len(ut)):
    ax[i].plot(ut[i][:50], ls='--')
    ax[i].plot(xt_trained[i][:50][:,10:20])
plt.tight_layout()

# full_procedure (in one step using trainingConfig)

In [ ]:
prng = np.random.default_rng(1001)

In [ ]:
N = 400
T_WASHOUT = 200
T_ADAPT = 500
T_LEARN = 5000 - 200 - 500
LOADING_REGULARIZER = 1e-3
APERTURE = 8

In [ ]:
trainingConfig = config.TrainingConfig(
    washout=T_WASHOUT,
    aperture=APERTURE,
    adapt = T_ADAPT,
    w_regularizer=LOADING_REGULARIZER,
    wout_regularizer=1e-3
)

In [ ]:
esnConfig = config.ESNConfig(
    input_size=1,
    reservoir_size= N,
    output_size=1,
    feedback=False,
    spectral_radius=1,
    init_weights='normal',
    init_weights_b='normal',
    init_weights_in='normal',
    init_weights_density=10/N,
    init_weights_in_density=1.0,
    init_weights__args={'loc': 0., 'scale': 1.},
    init_weights_b__args={'loc': 0., 'scale': 0.2},
    init_weights_in__args={'loc': 0., 'scale': 1.},
)

In [ ]:
T_pattern = 5000
ut = [
    #helper.n_periodic(5, T_pattern, np.random.default_rng(27)),
    #helper.n_periodic(5, T_pattern, np.random.default_rng(42)),
    helper.n_periodic_jong(0, 5, T_pattern),
    helper.n_periodic_jong(1, 5, T_pattern),
    helper.n_sine(8.83, T_pattern, phase=0),
    helper.n_sine(9.83, T_pattern, phase=0.)
]

In [ ]:
esn = ESN(esnConfig, prng)

In [ ]:
Ci = esn.full_procedure_diag(ut, trainingConfig)

In [ ]:
ut_zero = np.zeros_like(ut[0])
x_init = prng.uniform(low=-1., high=1., size=(esn.reservoir_size, 1))
xt_trained, yt_trained = list(zip(*[
    esn.harvest_states(ut_zero.copy(), x_init=x_init.copy(), C=Ci[i])
    for i in range(len(ut))
]))

In [ ]:
fig, ax = plt.subplots(len(ut), 1, figsize=(12, len(ut)), sharex=True, sharey=True)
for i in range(len(ut)):
    ax[i].plot(ut[i][50:90], ls='--')
    ax[i].plot(yt_trained[i][50:90])
plt.tight_layout()

# Hyperparameters search

In [ ]:
T_pattern = 5000
ut = [
    #helper.n_periodic(5, T_pattern, np.random.default_rng(27)),
    #helper.n_periodic(5, T_pattern, np.random.default_rng(42)),
    helper.n_periodic_jong(0, 5, T_pattern),
    helper.n_periodic_jong(1, 5, T_pattern),
    helper.n_sine(8.83, T_pattern, phase=0),
    helper.n_sine(9.83, T_pattern, phase=0.)
]

In [ ]:
N = 100
Winscal = [1.2,1.4,1]
SpecRadius = [0.6,1,1.4,2]
Apertj = [1,6,8,10,14]

b_scal = [0.1,0.2,0.4]
Wout_reg = [0,1e-1,1e-2,1e-3]
W_reg = [0,1e-1,1e-2,1e-3]


T_WASHOUT = 200
T_ADAPT = 500
T_LEARN = 5000 - 200 - 500

In [ ]:
L_RMSE = []
def hyperparasearch (Winscal, b_scal, Wout_reg, SpecRadius, Apertj, N, ut, W_reg, T_WASHOUT, T_ADAPT, T_LEARN):
    for b in b_scal:
        for win in Winscal:
            for rad in SpecRadius:
                for wout in Wout_reg:
                    for w in W_reg:
                        for aj in Apertj: 
                            prng = np.random.default_rng(1001)
                            print("aj: "+str(aj)+", b: "+str(b)+",wout:  "+str(wout)+", rad: "+str(rad), ", w:" + str(w) +
                                   ",win : "+str(win))
                            # Initialization
                            esnConfig = config.ESNConfig(
                                input_size=1,
                                reservoir_size= N,
                                output_size=1,
                                feedback=False,
                                spectral_radius=rad,
                                init_weights='normal',
                                init_weights_b='normal',
                                init_weights_in='normal',
                                init_weights_density=10/N,
                                init_weights_in_density=1.0,
                                init_weights__args={'loc': 0., 'scale': 1.},
                                init_weights_b__args={'loc': 0., 'scale': b},
                                init_weights_in__args={'loc': 0., 'scale': win},
                            )

                            trainingConfig = config.TrainingConfig(
                                washout=T_WASHOUT,
                                aperture=aj,
                                adapt = T_ADAPT,
                                w_regularizer=w,
                                wout_regularizer=wout
                            )

                            esn = ESN(esnConfig, prng)
                            Ci = esn.full_procedure(ut, trainingConfig)

                            ut_zero = np.zeros_like(ut[0])
                            x_init = prng.uniform(low=-1., high=1., size=(esn.reservoir_size, 1))
                            xt_trained, yt_trained = list(zip(*[
                                esn.harvest_states(ut_zero.copy(), x_init=x_init.copy(), C=Ci[i])
                                for i in range(len(ut))
                            ]))


                            fig, ax = plt.subplots(len(ut), 1, figsize=(12, len(ut)), sharex=True, sharey=True)
                            for i in range(len(ut)):
                                ax[i].plot(ut[i][50:90], ls='--')
                                ax[i].plot(yt_trained[i][50:90])
                            plt.tight_layout()
                            plt.show()
                            RMSE, _ = helper.testLRMSE(ut, yt_trained, 10)
                            L_RMSE.append([min(RMSE),"aj: "+str(aj)+", b: "+str(b)+",wout:  "+str(wout)+", rad: "+str(rad), ", w:" + str(w) +
                                   ",win : "+str(win)])

In [ ]:
hyperparasearch (Winscal, b_scal, Wout_reg, SpecRadius, Apertj, N, ut,W_reg, T_WASHOUT, T_ADAPT, T_LEARN)